In [1]:
import pandas as pd
import numpy as np
import sys
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.functions import *
from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col, mean, min, max
from pyspark.ml.linalg import Vectors
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression, DecisionTreeRegressor, GBTRegressor, RandomForestRegressor
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'

%load_ext autotime

In [3]:
# sc = SQLContext()
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
       .map(lambda line: line.split(','))
   #didn’t work with take(1). believe returns
   #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df


pandas_df = pd.read_csv('f_clusters6.csv')
df = sqlContext.createDataFrame(pandas_df)

#here’s the new vectorizer function:

def vectorizeData(data):
        return data.rdd.map(lambda r: [r[0], r[1], r[2], r[3], Vectors.dense(r[4:-1]),float(r[-1])])\
            .toDF(['route cluster', 'performance cluster', 'userid', 'workoutid', 'features', 'label'])

time: 3.67 s


In [4]:
df.schema

StructType(List(StructField(Unnamed: 0,LongType,true),StructField(route_prediction,LongType,true),StructField(perf_prediction,LongType,true),StructField(diff_altitude,DoubleType,true),StructField(geo_distance,DoubleType,true),StructField(userid,LongType,true),StructField(workoutid,LongType,true),StructField(heart_rate_avg,DoubleType,true),StructField(speed_avg,DoubleType,true),StructField(elapsed_time,LongType,true)))

time: 3.25 ms


In [5]:
select_columns = ['route_prediction', 'perf_prediction','userid', 'workoutid', 'geo_distance', 'diff_altitude', \
                'speed_avg', 'heart_rate_avg', 'elapsed_time']
df = df.select(select_columns)

time: 68.4 ms


In [6]:
df.schema

StructType(List(StructField(route_prediction,LongType,true),StructField(perf_prediction,LongType,true),StructField(userid,LongType,true),StructField(workoutid,LongType,true),StructField(geo_distance,DoubleType,true),StructField(diff_altitude,DoubleType,true),StructField(speed_avg,DoubleType,true),StructField(heart_rate_avg,DoubleType,true),StructField(elapsed_time,LongType,true)))

time: 22 ms


In [7]:
reg_df = vectorizeData(df)

time: 1.49 s


In [8]:
reg_df.schema

StructType(List(StructField(route cluster,LongType,true),StructField(performance cluster,LongType,true),StructField(userid,LongType,true),StructField(workoutid,LongType,true),StructField(features,VectorUDT,true),StructField(label,DoubleType,true)))

time: 2.38 ms


In [9]:
reg_df.rdd.count()

24642

time: 1.91 s


In [10]:
reg_df.count()

24642

time: 526 ms


In [11]:
reg_df

DataFrame[route cluster: bigint, performance cluster: bigint, userid: bigint, workoutid: bigint, features: vector, label: double]

time: 2.28 ms


In [12]:
reg_df.select('route cluster').rdd.distinct().count()

5

time: 625 ms


In [13]:
reg_df.groupBy(['route cluster','performance cluster']).count()\
    .orderBy(['route cluster', 'performance cluster']).show()

+-------------+-------------------+-----+
|route cluster|performance cluster|count|
+-------------+-------------------+-----+
|            0|                  0| 1903|
|            0|                  1| 4831|
|            0|                  2| 9150|
|            1|                  0|  342|
|            1|                  1| 1629|
|            1|                  2| 2151|
|            2|                  0|  414|
|            2|                  1| 1818|
|            2|                  2| 1764|
|            3|                  0|   36|
|            3|                  1|  175|
|            3|                  2|  295|
|            4|                  1|   80|
|            4|                  2|   54|
+-------------+-------------------+-----+

time: 896 ms


In [ ]:
reg_df[['route cluster','performance clsuter']]

In [14]:
reg_df.select(['route cluster','performance cluster']).distinct().collect()

[Row(route cluster=3, performance cluster=0),
 Row(route cluster=1, performance cluster=0),
 Row(route cluster=1, performance cluster=1),
 Row(route cluster=0, performance cluster=1),
 Row(route cluster=4, performance cluster=2),
 Row(route cluster=3, performance cluster=2),
 Row(route cluster=2, performance cluster=2),
 Row(route cluster=4, performance cluster=1),
 Row(route cluster=2, performance cluster=0),
 Row(route cluster=0, performance cluster=0),
 Row(route cluster=1, performance cluster=2),
 Row(route cluster=0, performance cluster=2),
 Row(route cluster=2, performance cluster=1),
 Row(route cluster=3, performance cluster=1)]

time: 475 ms


In [15]:
#distinct_clusters = reg_df.select('cluster').distinct().collect()
#cluster_numbers = [int(distinct_clusters[i][0]) for i in range(len(distinct_clusters))]

route_clusters = reg_df.select('route cluster').distinct().collect()
perf_clusters = reg_df.select('performance cluster').distinct().collect()
route_cluster_numbers = [int(route_clusters[i][0]) for i in range(len(route_clusters))]
perf_cluster_numbers = [int(perf_clusters[i][0]) for i in range(len(perf_clusters))]

time: 886 ms


In [16]:
reg_df.select('route cluster').distinct().collect()

[Row(route cluster=0),
 Row(route cluster=1),
 Row(route cluster=3),
 Row(route cluster=2),
 Row(route cluster=4)]

time: 313 ms


In [17]:
perf_cluster_numbers

[0, 1, 2]

time: 2.28 ms


In [18]:
d = reg_df[reg_df['route cluster']==3]
print d.count()
print d.select('performance cluster').distinct().collect()
d2 = d[d['performance cluster']==0]
d2.count()

506
[Row(performance cluster=0), Row(performance cluster=1), Row(performance cluster=2)]


36

time: 901 ms


In [19]:
reg_df[reg_df['performance cluster'] == 3].show()

+-------------+-------------------+------+---------+--------+-----+
|route cluster|performance cluster|userid|workoutid|features|label|
+-------------+-------------------+------+---------+--------+-----+
+-------------+-------------------+------+---------+--------+-----+

time: 377 ms


In [20]:
reg_df.rdd.count()

24642

time: 444 ms


In [21]:
#add two
for n in perf_cluster_numbers:
    for i in route_cluster_numbers:
            print("(Route, Perf) Cluster Pair: " + str((n,i)))
            temp_df1 = reg_df[reg_df['route cluster']==i]
            temp_df = temp_df1[reg_df['performance cluster']==n]
            
            if temp_df.count() == 0:
                print "Cluster pair dropped"
                print "\n\n"
                pass
            else:
                temp_lr = LinearRegression()
                #temp_df.show(5)
                temp_lrModel = temp_lr.fit(temp_df['label','features'])
                temp_df = temp_lrModel.transform(temp_df)

                #Print the coefficients and intercept for linear regression
                print("Cofficient Names: " + 'sum_geo_distance, diff_altitude, new_avg_speed, and new_avg_dist')
                print("Coefficients: " + str(temp_lrModel.coefficients))
                print("Intercept: " + str(temp_lrModel.intercept))

                # Summarize the model over the training set and print out some metrics
                trainingSummary = temp_lrModel.summary
                print("numIterations: %d" % trainingSummary.totalIterations)
                print("objectiveHistory: %s" % str(trainingSummary.objectiveHistory))
                trainingSummary.residuals.show()
                print("RMSE: %f" % trainingSummary.rootMeanSquaredError)
                print("r2: %f" % trainingSummary.r2)
                print("\n\n")

(Route, Perf) Cluster Pair: (0, 0)
Cofficient Names: sum_geo_distance, diff_altitude, new_avg_speed, and new_avg_dist
Coefficients: [13767.7299132,-5493.35880407,-381.903510974,-0.822899247926]
Intercept: 7901.35281739
numIterations: 1
objectiveHistory: [0.0]
+-------------------+
|          residuals|
+-------------------+
| -629.9844780102812|
|-276.02662983343726|
|-125.37377866523275|
| -838.7108867512316|
|  186.4439517303099|
| -695.7664456845296|
|  -312.377185852959|
| 2488.4190254294026|
| 1473.7085458678448|
|  234.7716115522844|
| -866.7419970147221|
| 315.12692251756425|
| -393.6958035329826|
|-224.72746372736492|
| -966.8309479989066|
| -363.4450093438809|
| 3912.5576837758163|
| 131.66699439077274|
| -648.4994352789081|
|-180.68216532177848|
+-------------------+
only showing top 20 rows

RMSE: 5708.407586
r2: 0.052187



(Route, Perf) Cluster Pair: (0, 1)
Cofficient Names: sum_geo_distance, diff_altitude, new_avg_speed, and new_avg_dist
Coefficients: [14684.198449,-431.0